In [ ]:
# Import dependencies
try:
    import openpyxl
except ImportError:
    %pip install openpyxl
    import openpyxl

import datetime
import pandas as pd
from pyspark.sql import functions as F

# -----------------------
# 1️⃣ Load Excel file
# -----------------------
file_path = "/Workspace/Users/financial-analytics-hub/spend_by_region.xlsx"
raw_df = pd.read_excel(file_path, header=None)

fiscal_year_row = raw_df.iloc[5]
fiscal_month_row = raw_df.iloc[7]
new_columns = [f"{fy}_{fm}" for fy, fm in zip(fiscal_year_row, fiscal_month_row)]

df = raw_df.iloc[8:].copy()
df.columns = new_columns
df.reset_index(drop=True, inplace=True)
df.columns.values[0] = "Region"
df.columns.values[1] = "Account"
df = df.iloc[:, :-5]

melted = df.melt(id_vars=["Region", "Account"], var_name="Fiscal", value_name="Amount")
melted[['FiscalYear', 'FiscalMonth']] = melted['Fiscal'].str.split('_', expand=True)
melted.drop(columns='Fiscal', inplace=True)
melted['Region'] = melted['Region'].str.strip()
melted['Account'] = melted['Account'].str.strip()

# -----------------------
# 2️⃣ Process Field & hospital Sales
# -----------------------
accounts_to_keep = ["TOTAL EXPENSES", "TOTAL EMPLOYEE TRAVEL"]

def process_sales(region_prefix, final_label):
    regions_to_keep = [
        f"{region_prefix}-Region 1",
        f"{region_prefix}-Region 2",
        f"{region_prefix}-Region 3",
        f"{region_prefix}-Region 4",
        f"{region_prefix}-Region 5"
    ]
    
    region_rename_map = {
        f"{region_prefix}-Region 1": "Region 1",
        f"{region_prefix}-Region 2": "Region 2",
        f"{region_prefix}-Region 3": "Region 3",
        f"{region_prefix}-Region 4": "Region 5",
        f"{region_prefix}-Region 5": "Region 5"
    }
    
    filtered_df = melted[melted["Account"].isin(accounts_to_keep)]
    filtered_regions_df = filtered_df[filtered_df["Region"].isin(regions_to_keep)].copy()
    filtered_regions_df["Region"] = filtered_regions_df["Region"].replace(region_rename_map)
    
    pivoted = filtered_regions_df.pivot_table(
        index=["Region", "FiscalYear", "FiscalMonth"],
        columns="Account",
        values="Amount",
        aggfunc="sum"
    ).reset_index().fillna(0)
    
    pivoted = pivoted.rename(columns={
        "TOTAL EXPENSES": final_label,
        "TOTAL EMPLOYEE TRAVEL": "Travel & Entertainment"
    })
    
    pivoted[f"{final_label} (Adj)"] = pivoted[final_label] - pivoted["Travel & Entertainment"]
    pivoted = pivoted.drop(columns=[final_label])
    
    final_df = pivoted.melt(
        id_vars=["Region", "FiscalYear", "FiscalMonth"],
        value_vars=[f"{final_label} (Adj)", "Travel & Entertainment"],
        var_name="Category",
        value_name="Amount"
    )
    
    final_df["Category"] = final_df["Category"].replace({f"{final_label} (Adj)": final_label})
    return final_df

field_sales_df = process_sales("Field", "Field Sales")
hospital_sales_df = process_sales("hospital", "hospital Sales")

spend_df = pd.concat([field_sales_df, hospital_sales_df], ignore_index=True)

spend_df["FiscalYear"] = spend_df["FiscalYear"].str.replace("FY", "20").astype(int)

import datetime

# Step 1: Map fiscal month names to numbers
month_map = {
    "May": 4, "June": 5, "July": 6, "August": 7, "September": 8,
    "October": 9, "November": 10, "December": 11,
    "January": 12, "February": 1, "March": 2, "April": 3
}

# Step 2: Convert month names to numbers and recalculate quarter
spend_df["FiscalMonthNumber"] = spend_df["FiscalMonth"].map(month_map)
spend_df = spend_df.drop(columns=["FiscalMonth"])
spend_df = spend_df.rename(columns={"FiscalMonthNumber": "FiscalMonth"})
spend_df["FiscalQuarter"] = ((spend_df["FiscalMonth"] - 1) // 3) + 1

def fiscal_to_calendar_date(fiscal_year, fiscal_month):
    calendar_month = ((fiscal_month + 7 - 1) % 12) + 1
    calendar_year = fiscal_year if fiscal_month >= 5 else fiscal_year - 1
    return datetime.date(calendar_year, calendar_month, 15)

spend_df["Date"] = spend_df.apply(lambda row: fiscal_to_calendar_date(row["FiscalYear"], row["FiscalMonth"]), axis=1)
spend_df = spend_df[["Region", "FiscalYear", "FiscalQuarter", "FiscalMonth", "Date", "Category", "Amount"]]

spend_spark_df = spark.createDataFrame(spend_df)

# -----------------------
# 3️⃣ SQL Query for SalesPct
# -----------------------
sales_pct_df = spark.sql("""
WITH RegionalRevenue AS (
    SELECT Region, FiscalYear, FiscalMonth, SUM(Amount) AS Revenue
    FROM analytics.gold.revenue_by_region
    WHERE Category = 'Revenue'
    GROUP BY Region, FiscalYear, FiscalMonth
),
TotalRevenue AS (
    SELECT FiscalYear, FiscalMonth, SUM(Amount) AS TotalRevenue
    FROM analytics.gold.revenue_by_region
    WHERE Category = 'Revenue'
    GROUP BY FiscalYear, FiscalMonth
)
SELECT r.Region, r.FiscalYear, r.FiscalMonth, r.Revenue, t.TotalRevenue,
CAST(r.Revenue AS DOUBLE) / CAST(t.TotalRevenue AS DOUBLE) AS SalesPct
FROM RegionalRevenue r
JOIN TotalRevenue t
  ON r.FiscalYear = t.FiscalYear AND r.FiscalMonth = t.FiscalMonth
""")

# -----------------------
# 4️⃣ alloc1 & alloc2 Allocations
# -----------------------
alloc1_total_df = melted[(melted["Account"] == "TOTAL EXPENSES") & (melted["Region"] == "alloc1_cc")]
alloc1_total_df = alloc1_total_df.groupby(['FiscalYear', 'FiscalMonth'], as_index=False)['Amount'].sum()

alloc2_total_df = melted[(melted["Account"] == "TOTAL EXPENSES") & (melted["Region"] == "alloc2_cc")]
alloc2_total_df = alloc2_total_df.groupby(['FiscalYear', 'FiscalMonth'], as_index=False)['Amount'].sum()

alloc1_total_spark = spark.createDataFrame(alloc1_total_df).withColumn(
    "FiscalYear", F.regexp_replace(F.col("FiscalYear"), "FY", "20").cast("int")
).withColumn(
    "FiscalMonth", F.create_map(
        *[F.lit(x) for x in [
            "May", 4, "June", 5, "July", 6, "August", 7, "September", 8,
            "October", 9, "November", 10, "December", 11, "January", 12,
            "February", 1, "March", 2, "April", 3
        ]]
    )[F.col("FiscalMonth")]
)

alloc2_total_spark = spark.createDataFrame(alloc2_total_df).withColumn(
    "FiscalYear", F.regexp_replace(F.col("FiscalYear"), "FY", "20").cast("int")
).withColumn(
    "FiscalMonth", F.create_map(
        *[F.lit(x) for x in [
            "May", 4, "June", 5, "July", 6, "August", 7, "September", 8,
            "October", 9, "November", 10, "December", 11, "January", 12,
            "February", 1, "March", 2, "April", 3
        ]]
    )[F.col("FiscalMonth")]
)

alloc1_total_spark = alloc1_total_spark.withColumnRenamed("Amount", "alloc1Total")
alloc2_total_spark = alloc2_total_spark.withColumnRenamed("Amount", "alloc2Total")

alloc1_df = sales_pct_df.join(
    alloc1_total_spark, ["FiscalYear", "FiscalMonth"], "inner"
).withColumn("Amount", F.col("alloc1Total") * F.col("SalesPct")).withColumn("Category", F.lit("Allocations 2")).select("Region", "FiscalYear", "FiscalMonth", "Category", "Amount")

alloc2_df = sales_pct_df.join(
    alloc2_total_spark, ["FiscalYear", "FiscalMonth"], "inner"
).withColumn("Amount", F.col("alloc2Total") * F.col("SalesPct")).withColumn("Category", F.lit("Allocations 1")).select("Region", "FiscalYear", "FiscalMonth", "Category", "Amount")

alloc_df = alloc1_df.unionByName(alloc2_df).withColumn(
    "FiscalQuarter", ((F.col("FiscalMonth") - 1) / 3 + 1).cast("int")
).withColumn(
    "Date", F.udf(lambda fy,fm: fiscal_to_calendar_date(fy,fm), "date")(F.col("FiscalYear"), F.col("FiscalMonth"))
).select("Region", "FiscalYear", "FiscalQuarter", "FiscalMonth", "Date", "Category", "Amount")

# -----------------------
# 5️⃣ Std Cost Query
# -----------------------
stdcost_query = """
CREATE OR REPLACE TABLE analytics.gold.revenue_by_region AS

  SELECT
  region_lookup.region_name AS Region,
  date.FiscalYear,
  date.FiscalQuarter,
  date.FiscalMonth,
  CAST(date.date AS DATE) AS Date,
  'Revenue' AS Category,
  CAST(sales_data.net_sales_credit_u AS INT) AS UnitsShipped
FROM analytics.gold.sales_data AS sales_data
INNER JOIN analytics.gold.date AS date
  ON sales_data.sales_credit_date = date.date
INNER JOIN analytics.gold.region_lookup AS region_lookup
  ON sales_data.office_id = region_lookup.office_id
WHERE
  sales_data.sales_force_id = 1
  AND region_lookup.sales_force_id = 1
  AND sales_data.sales_cycle_step_id = 12
  AND sales_data.fiscal_yr_id IN (2025, 2026, 2027)
  AND sales_data.sales_plan_type_name = 'Total Rev';
"""
units_df = spark.sql(stdcost_query)

StandardUnitCost_pd_df = pd.read_excel("/Workspace/Users/financial-analytics-hub/Std Cost StandardUnitCost.xlsx")
StandardUnitCost_pd_trimmed = StandardUnitCost_pd_df[['Material', 'FY25 StandardUnitCost']].rename(columns={'Material': 'ProductNumber', 'FY25 StandardUnitCost': 'StandardUnitCost'})
StandardUnitCost_spark_df = spark.createDataFrame(StandardUnitCost_pd_trimmed)

std_cost_df = units_df.join(StandardUnitCost_spark_df, on='ProductNumber', how='left').withColumn(
    'Amount', F.col('UnitsShipped') * F.col('StandardUnitCost')
).withColumn(
    'Category', F.lit('Std Cost')
).filter(
    (F.col('StandardUnitCost').isNotNull()) & (F.col('UnitsShipped') != 0)
).select("Region", "FiscalYear", "FiscalQuarter", "FiscalMonth", "Date", "Category", "Amount")

# -----------------------
# 6️⃣ UNION ALL → FINAL
# -----------------------
final_df = spend_spark_df.unionByName(alloc_df).unionByName(std_cost_df)

# -----------------------
# 7️⃣ Write final combined table
# -----------------------
spark.sql("DROP TABLE IF EXISTS analytics.gold.spend_by_region")

final_df.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable(analytics.gold.spend_by_region")

print("✅ Combined spend + allocations + std cost saved to analytics.gold.spend_by_region")
